In [2]:
import xarray as xr
import matplotlib.pyplot as plt
from scipy.stats import theilslopes
import cartopy.crs as ccrs
import matplotlib as mpl
import numpy as np
from pyclim_noresm.general_util_funcs import global_avg,mask_region_latlon
import pandas as pd
from statsmodels.tsa.tsatools import detrend, add_trend
import statsmodels.tsa.api as tsa
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.tsa.stattools import adfuller


In [4]:
dsets_histSST = [xr.open_dataset(p) for p in snakemake.input.histSST]
dsets_histSST = {f'{ds.source_id}_{ds.variable_id}' : ds for ds in dsets_histSST}
dsets_piClim_aer = [xr.open_dataset(p) for p in snakemake.input.piClim_aer]
dsets_piClim_aer = {f'{ds.source_id}_{ds.variable_id}' : ds for ds in dsets_piClim_aer}
forcing_estimates = pd.read_csv('workflow/input_data/Smith_forcing_trends.csv',index_col=0)

In [5]:
forcing_regions = {
    'East China': {'lon0':80, 'lat0':25, 'lon1':145,'lat1':50},
    'India'     : {'lon0':63, 'lat0':5,'lon1':93,'lat1':30},
    'Europe'    : {'lon0':-5, 'lat0':35, 'lon1':40,'lat1':65},
    'North America': {'lon0':-100, 'lat0': 23, 'lon1':-50, 'lat1':53},
    'Global'       : {'lon0':None, 'lat0': None, 'lon1': None, 'lat1':None}
                  }

In [6]:
fig ,ax = plt.subplots(figsize=(12,7), subplot_kw={'projection': ccrs.Robinson()})
ax.set_global()
ax.add_patch(mpl.patches.Rectangle(xy=[80,25],width=65,height=25, transform=ccrs.PlateCarree(),
                                   fill=False, edgecolor='tab:blue', linewidth=2))

ax.add_patch(mpl.patches.Rectangle(xy=[-5,35],width=50,height=30, transform=ccrs.PlateCarree(),
                                   fill=False, edgecolor='tab:orange', linewidth=2))

ax.add_patch(mpl.patches.Rectangle(xy=[-100,23],width=50,height=30, transform=ccrs.PlateCarree(),
                                   fill=False, edgecolor='tab:purple', linewidth=2))

ax.add_patch(mpl.patches.Rectangle(xy=[63,5],width=30,height=25, transform=ccrs.PlateCarree(),
                                   fill=False, edgecolor='tab:green', linewidth=2))


ax.coastlines()
# ax.set_xticks([-180,-140,-100,-60, -20, 20 , 60, 100, 140, 180], crs=ccrs.PlateCarree())
# ax.set_yticks([-90,-70,-50,-30,-10,10,30,50,70,90])
gl = ax.gridlines()
gl.left_labels=True
gl.bottom_labels=True
# gl.xlocator(10)


In [59]:
def regional_create_time_series(dsets: dict, region_dict:dict, norm=False, standardize=False,hist=True):
    out_dict = {}
    for source_id, ds in dsets.items():
        for region, c in region_dict.items():
            shift=0
            if ds.lon.max() > 356:
                shift = 180 
            if region == 'Global':
                temp_ds = ds[ds.variable_id]
            else:
                
                temp_ds = ds[ds.variable_id].sel(lon=slice(c['lon0']+shift, c['lon1']+shift), lat=slice(c['lat0'],c['lat1']))
            if 'year' in temp_ds.dims:
                temp_ds = temp_ds.rename({'year':'time'})
            if temp_ds.time[0].dtype == 'O':
                temp_ds = temp_ds.assign_coords(time=temp_ds.indexes['time'].to_datetimeindex())
            temp_ds = global_avg(temp_ds)
            
            temp_ds = temp_ds
            if norm:
                temp_ds = (temp_ds-temp_ds.min(axis=0)/(temp_ds.max(axis=0)-temp_ds.min(axis=0)))
            elif standardize:
                temp_ds = (temp_ds-temp_ds.mean(axis=0)/(temp_ds.std(axis=0)))
            if hist:
                out_dict[f'{region}_{source_id}'] = temp_ds.to_pandas()
            else:
                temp_ds = temp_ds.to_pandas().reset_index()
                out_dict[f'{region}_{source_id}'] = temp_ds.drop(columns=['time'])[0]
    if hist==False:
        index = temp_ds.index
        df = pd.DataFrame(out_dict, index=index, columns=out_dict.keys())
    else:
        df = pd.DataFrame(out_dict)
    return df


def _testStationary(df, key=None, resamplingAvg=None, confidence_level=0.95):
    if resamplingAvg != None:
        df = df.resample(resamplingAvg).mean()

    print(" > Is the data stationary ?")
    try:
        if key==None:
            key = df.keys()[0]
        dftest = adfuller(df[key].dropna(), autolag='AIC')
    except AttributeError:
        dftest = adfuller(df,autolag='AIC')
    print("Test statistic = {:.3f}".format(dftest[0]))
    print("P-value = {:.3f}".format(dftest[1]))
    print("Critical values :")
    for k, v in dftest[4].items():
        print("\t{}: {} - The data is {} stationary with {}% confidence".format(k, v, "not" if v<dftest[0] else "", 100-int(k[:-1])))


def add_linear_trends(df, a=1, c=0):
    trend = add_trend(df, 'ct')
    t = trend['trend']*a
    const = c
    trend = trend.add(t+const, axis=0)
    trend['const'] = const 
    trend['trend'] = t
    
    return trend

def MK_test(x ,verbose=True):
    n = len(x)
    S = 0
    for i in range(n-1):
        for j in range(i+1, n):
            S += np.sign(x[j] - x[i])

    # number of tied groups
    q = len(np.unique(x)) # number of unique values

    # number of ties for the p-th value
    tp = np.zeros(np.unique(x).shape)
    for i in range(q):
        tp[i] = sum(x == np.unique(x)[i])

    var_s = (n*(n-1)*(2*n+5) - np.sum(tp*(tp-1)*(2*tp+5)))/18

    # calculate the MK test statistic Z_MK
    if S > 0:
        Z_MK = (S - 1)/np.sqrt(var_s)
    elif S < 0:
        Z_MK = (S + 1)/np.sqrt(var_s)
    else:
        Z_MK = 0

    # calculate the p_value
    p = 2*(1-norm.cdf(abs(Z_MK))) #two tailed test
    if verbose:
        print("sample size="+str(n))
        print("S="+str(S))
        print("var(S)="+str(var_s))
        print("Z_MK="+str(Z_MK))
        print("p-value="+str(p))

    return p

def rolling_detrend(df, windowsize=3):
    return df-df.rolling(windowsize, center=True).mean()/df.rolling(windowsize, center=True).std()


In [93]:
def artificial_trend_analysis(df: pd.DataFrame, 
                              trend: float, 
                              region : str = 'Global', 
                              intercept: float = 0.0, hist=True):
    vname = df.columns[0].split('_')[-1]
    models = list(set([c.split('_')[1] for c in df.columns]))

    if hist:
        detrended = df.diff()
        var_PI = detrended.loc['1860':'1890']
        var_1950 = detrended.loc['1950':'1980']
        var_1980 = detrended.loc['1980':'2010']
    else:
        var_PI = df.iloc[5:30,:] - df.iloc[5:30,:].mean()
    trend_pi = add_linear_trends(var_PI, trend) 
    if hist:
        trend_1950 = add_linear_trends(var_1950, forcing_estimates.loc['CMIP6-constrained','mean']) 
        trend_1980 = add_linear_trends(var_1980, forcing_estimates.loc['CMIP6-constrained','mean']) 
    out_df = pd.DataFrame(columns=models+ ['True trend'], index=['PI','1950','1980'])
    significance =  pd.DataFrame(columns=models, index=['PI','1950','1980'])
    out_df['True trend'] = trend 
    for model in models:
        X = sm.add_constant(np.arange(0,len(trend_pi[f'{region}_{model}_{vname}'])))

        result_pi = sm.OLS(trend_pi[f'{region}_{model}_{vname}'],X).fit()
        significance.loc['PI',model] = MK_test(trend_pi[f'{region}_{model}_{vname}'].values, verbose=False) < 0.05

        out_df.loc['PI',model] = result_pi.params[1]
        if hist:
            result_1950 = sm.OLS(trend_1950[f'{region}_{model}_{vname}'],X).fit()
            result_1980 = sm.OLS(trend_1980[f'{region}_{model}_{vname}'],X).fit()
            significance.loc['1950',model] = MK_test(trend_1950[f'{region}_{model}_{vname}'].values, verbose=False) < 0.05
            significance.loc['1980',model] = MK_test(trend_1980[f'{region}_{model}_{vname}'].values, verbose=False) < 0.05
            out_df.loc['1950',model] = result_1950.params[1]
            out_df.loc['1980',model] = result_1980.params[1]
        


    return out_df, significance
    

In [60]:
df_pi = regional_create_time_series(dsets_piClim_aer, forcing_regions, standardize=False, hist=False)

In [26]:
df = regional_create_time_series(dsets_histSST, forcing_regions, standardize=False)
df_pi = regional_create_time_series(dsets_piClim_aer, forcing_regions, standardize=False, hist=False)
detrended = df.diff()
trenddf ,r = artificial_trend_analysis(df, forcing_estimates.loc['CMIP6-constrained','mean'])
var_PI = detrended.loc['1860':'1890']
var_1950 = detrended.loc['1950':'1980']
var_1980 = rolling_detrend(df.loc['1980':'2010'])
# detrended_PI = df.loc['1850':'1880']

In [84]:
trenddf_pi ,r_pi = artificial_trend_analysis(df_pi, forcing_estimates.loc['CMIP6-constrained','mean']*-20, hist=False, region='Europe')

In [99]:
artificial_trend_analysis(df_pi, forcing_estimates.loc['CMIP6-constrained','mean']*5,intercept=10, hist=False, region='Europe')

In [69]:
detrended_pi = df_pi-df_pi.mean()[

In [97]:
detrended_pi['Europe_NorESM2-LM_ERFt'].plot()

In [98]:
var_PI['Europe_NorESM2-LM_ERFt'].plot()

In [96]:
trenddf_pi

In [86]:
trenddf_pi

In [87]:
r_pi

In [75]:
r_pi

In [218]:
rolling_dt.loc['1860':'1890']['Global_NorESM2-LM_ERFt'].plot()

In [139]:
models = [c.split('_')[1] for c in df.columns]

In [141]:
df.columns

In [133]:
list(set(models))

In [73]:
(trenddf['East China_NorESM2-LM_ERFt']).plot()
var_1950['East China_NorESM2-LM_ERFt'].plot

In [76]:
var_1950['East China_NorESM2-LM_ERFt'].mean()

In [97]:
model.params[1]

In [98]:
-0.5*forcing_estimates.loc['CMIP6-constrained','mean']

In [104]:
np.abs(model.params[1])/(forcing_estimates.loc['CMIP6-constrained','mean'])*100

In [103]:
X = sm.add_constant(np.arange(0,len(trenddf['Europe_NorESM2-LM_ERFt'])))

model = sm.OLS(trenddf['East China_NorESM2-LM_ERFt'],X).fit()

line = model.params[0] + model.params[1]*np.arange(0,len(trenddf['East China_NorESM2-LM_ERFt'].index))
trenddf['East China_NorESM2-LM_ERFt'].plot(marker='.', markersize=10, linestyle='')
plt.plot(trenddf['trend']+model.params[0])
plt.plot(trenddf['East China_NorESM2-LM_ERFt'].index, line)

In [39]:
line.sum()

In [34]:
(trenddf['trend']+trenddf['const']).sum()

In [206]:
X = sm.add_constant(np.arange(0,len(trenddf['Europe_NorESM2-LM_ERFt'])))

model = sm.OLS(trenddf['Europe_NorESM2-LM_ERFt'],X).fit()

line = model.params[0] + model.params[1]*np.arange(0,len(trenddf['Europe_NorESM2-LM_ERFt'].index))

In [207]:
model.params[1]*np.arange(0,len(trenddf['Europe_NorESM2-LM_ERFt'].index)).sum()

In [208]:
trenddf['trend'].sum()

In [209]:
trenddf['Europe_NorESM2-LM_ERFt'].plot(marker='.', markersize=10, linestyle='')
plt.plot(trenddf['trend']+model.params[0], label='actual trend')
plt.plot(trenddf['East China_NorESM2-LM_ERFt'].index, line, label='estimated trend')
plt.legend()

In [157]:
def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)

In [159]:
adf_test(var_1950['East China_NorESM2-LM_ERFt'])